In [1]:
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res[-126:])
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
X = np.array(sequences)

In [10]:
X.shape

(700, 30, 126)

In [11]:
y = to_categorical(labels).astype(int)

In [13]:
loss = []
accuracies = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = X[train_index], X[test_index]
    labels_train, labels_test = y[train_index], y[test_index]
    
    # Construct the required LSTM model.
    model = Sequential() # changed activation to tanh
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['categorical_accuracy',Precision(),Recall()]) 
    
    model_training_history = model.fit(features_train, labels_train, epochs=300, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

    # Evaluate the trained model.
    model_evaluation_history = model.evaluate(features_test, labels_test)
    
    eval_loss, eval_acc, eval_prec, eval_rec = model_evaluation_history
    loss.append(eval_loss)
    accuracies.append(eval_acc)
    precisions.append(eval_prec)
    recalls.append(eval_rec)

Model Created Successfully!
Epoch 1/300
112/112 [==============================] - 13s 57ms/step - loss: 2.4883 - categorical_accuracy: 0.1183 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.3682 - val_categorical_accuracy: 0.1071 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/300
112/112 [==============================] - 5s 41ms/step - loss: 2.1978 - categorical_accuracy: 0.1875 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.2056 - val_categorical_accuracy: 0.0804 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/300
112/112 [==============================] - 5s 43ms/step - loss: 2.0758 - categorical_accuracy: 0.2299 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.0541 - val_categorical_accuracy: 0.2054 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/300
112/112 [==============================] - 5s 42ms/step - loss: 1.8682 - categorical_accuracy: 0.3460 - precision_1: 0.0000e+00 - recall_

112/112 [==============================] - 6s 54ms/step - loss: 0.2525 - categorical_accuracy: 0.9554 - precision_1: 0.9703 - recall_1: 0.9464 - val_loss: 0.2310 - val_categorical_accuracy: 0.9643 - val_precision_1: 0.9722 - val_recall_1: 0.9375
Epoch 33/300
112/112 [==============================] - 7s 58ms/step - loss: 0.2463 - categorical_accuracy: 0.9442 - precision_1: 0.9522 - recall_1: 0.9330 - val_loss: 0.2949 - val_categorical_accuracy: 0.9375 - val_precision_1: 0.9429 - val_recall_1: 0.8839
Epoch 34/300
112/112 [==============================] - 12s 105ms/step - loss: 0.2237 - categorical_accuracy: 0.9598 - precision_1: 0.9594 - recall_1: 0.9487 - val_loss: 0.2028 - val_categorical_accuracy: 0.9821 - val_precision_1: 0.9818 - val_recall_1: 0.9643
Epoch 35/300
112/112 [==============================] - 8s 70ms/step - loss: 0.1617 - categorical_accuracy: 0.9844 - precision_1: 0.9888 - recall_1: 0.9821 - val_loss: 0.1658 - val_categorical_accuracy: 0.9821 - val_precision_1: 0.990

Epoch 64/300
112/112 [==============================] - 4s 39ms/step - loss: 0.0826 - categorical_accuracy: 0.9821 - precision_1: 0.9865 - recall_1: 0.9777 - val_loss: 0.1107 - val_categorical_accuracy: 0.9464 - val_precision_1: 0.9636 - val_recall_1: 0.9464
Epoch 65/300
112/112 [==============================] - 5s 44ms/step - loss: 0.0473 - categorical_accuracy: 0.9933 - precision_1: 0.9933 - recall_1: 0.9933 - val_loss: 0.0777 - val_categorical_accuracy: 0.9821 - val_precision_1: 0.9818 - val_recall_1: 0.9643
Epoch 66/300
112/112 [==============================] - 5s 41ms/step - loss: 0.0448 - categorical_accuracy: 0.9933 - precision_1: 0.9955 - recall_1: 0.9933 - val_loss: 0.0856 - val_categorical_accuracy: 0.9732 - val_precision_1: 0.9817 - val_recall_1: 0.9554
Epoch 67/300
112/112 [==============================] - 4s 40ms/step - loss: 0.0429 - categorical_accuracy: 0.9933 - precision_1: 0.9933 - recall_1: 0.9933 - val_loss: 0.0992 - val_categorical_accuracy: 0.9643 - val_precisi

Epoch 12/300
112/112 [==============================] - 7s 63ms/step - loss: 0.9615 - categorical_accuracy: 0.6920 - precision_2: 0.9026 - recall_2: 0.3103 - val_loss: 0.9120 - val_categorical_accuracy: 0.7054 - val_precision_2: 0.8857 - val_recall_2: 0.2768
Epoch 13/300
112/112 [==============================] - 6s 51ms/step - loss: 0.9344 - categorical_accuracy: 0.6875 - precision_2: 0.9040 - recall_2: 0.3571 - val_loss: 0.8123 - val_categorical_accuracy: 0.7143 - val_precision_2: 0.8814 - val_recall_2: 0.4643
Epoch 14/300
112/112 [==============================] - 7s 63ms/step - loss: 0.8279 - categorical_accuracy: 0.7299 - precision_2: 0.9202 - recall_2: 0.4888 - val_loss: 0.8161 - val_categorical_accuracy: 0.7232 - val_precision_2: 0.9464 - val_recall_2: 0.4732
Epoch 15/300
112/112 [==============================] - 7s 61ms/step - loss: 0.7901 - categorical_accuracy: 0.7589 - precision_2: 0.9587 - recall_2: 0.5179 - val_loss: 0.7454 - val_categorical_accuracy: 0.7768 - val_precisi

112/112 [==============================] - 6s 53ms/step - loss: 0.1989 - categorical_accuracy: 0.9531 - precision_2: 0.9652 - recall_2: 0.9286 - val_loss: 0.1953 - val_categorical_accuracy: 0.9554 - val_precision_2: 0.9554 - val_recall_2: 0.9554
Epoch 44/300
112/112 [==============================] - 7s 64ms/step - loss: 1.0345 - categorical_accuracy: 0.6607 - precision_2: 0.7735 - recall_2: 0.5871 - val_loss: 0.5100 - val_categorical_accuracy: 0.8393 - val_precision_2: 0.8958 - val_recall_2: 0.7679
Epoch 45/300
112/112 [==============================] - 6s 56ms/step - loss: 0.3561 - categorical_accuracy: 0.8996 - precision_2: 0.9369 - recall_2: 0.8616 - val_loss: 0.3232 - val_categorical_accuracy: 0.9018 - val_precision_2: 0.9245 - val_recall_2: 0.8750
Epoch 46/300
112/112 [==============================] - 7s 59ms/step - loss: 0.2963 - categorical_accuracy: 0.9040 - precision_2: 0.9211 - recall_2: 0.8862 - val_loss: 0.2736 - val_categorical_accuracy: 0.9196 - val_precision_2: 0.9252 

...

Epoch 73/300
112/112 [==============================] - 8s 73ms/step - loss: 0.0316 - categorical_accuracy: 0.9978 - precision_4: 0.9978 - recall_4: 0.9955 - val_loss: 0.1031 - val_categorical_accuracy: 0.9821 - val_precision_4: 0.9908 - val_recall_4: 0.9643
Epoch 74/300
112/112 [==============================] - 8s 73ms/step - loss: 0.0287 - categorical_accuracy: 0.9978 - precision_4: 0.9978 - recall_4: 0.9978 - val_loss: 0.1025 - val_categorical_accuracy: 0.9732 - val_precision_4: 0.9909 - val_recall_4: 0.9732
Epoch 75/300
112/112 [==============================] - 8s 70ms/step - loss: 0.0280 - categorical_accuracy: 0.9955 - precision_4: 0.9978 - recall_4: 0.9955 - val_loss: 0.1033 - val_categorical_accuracy: 0.9732 - val_precision_4: 0.9909 - val_recall_4: 0.9732
Epoch 76/300
112/112 [==============================] - 8s 70ms/step - loss: 0.0266 - categorical_accuracy: 0.9978 - precision_4: 0.9978 - recall_4: 0.9955 - val_loss: 0.1044 - val_categorical_accuracy: 0.9732 - val_precisi

Epoch 18/300
112/112 [==============================] - 9s 84ms/step - loss: 0.5928 - categorical_accuracy: 0.8527 - precision_5: 0.9282 - recall_5: 0.7210 - val_loss: 0.6468 - val_categorical_accuracy: 0.8214 - val_precision_5: 0.9221 - val_recall_5: 0.6339
Epoch 19/300
112/112 [==============================] - 9s 77ms/step - loss: 0.5688 - categorical_accuracy: 0.8571 - precision_5: 0.9359 - recall_5: 0.7500 - val_loss: 0.7330 - val_categorical_accuracy: 0.8304 - val_precision_5: 0.8816 - val_recall_5: 0.5982
Epoch 20/300
112/112 [==============================] - 8s 71ms/step - loss: 0.4940 - categorical_accuracy: 0.8906 - precision_5: 0.9536 - recall_5: 0.7790 - val_loss: 0.5460 - val_categorical_accuracy: 0.8750 - val_precision_5: 0.8925 - val_recall_5: 0.7411
Epoch 21/300
112/112 [==============================] - 8s 72ms/step - loss: 0.4601 - categorical_accuracy: 0.8884 - precision_5: 0.9399 - recall_5: 0.8036 - val_loss: 0.5062 - val_categorical_accuracy: 0.8839 - val_precisi

Epoch 50/300
112/112 [==============================] - 8s 74ms/step - loss: 0.0960 - categorical_accuracy: 0.9844 - precision_5: 0.9865 - recall_5: 0.9799 - val_loss: 0.1717 - val_categorical_accuracy: 0.9554 - val_precision_5: 0.9640 - val_recall_5: 0.9554
Epoch 51/300
112/112 [==============================] - 8s 69ms/step - loss: 0.0758 - categorical_accuracy: 0.9888 - precision_5: 0.9910 - recall_5: 0.9844 - val_loss: 0.0996 - val_categorical_accuracy: 0.9821 - val_precision_5: 0.9908 - val_recall_5: 0.9643
Epoch 52/300
112/112 [==============================] - 8s 72ms/step - loss: 0.0557 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 0.9978 - val_loss: 0.1021 - val_categorical_accuracy: 0.9821 - val_precision_5: 0.9910 - val_recall_5: 0.9821
Epoch 53/300
112/112 [==============================] - 8s 71ms/step - loss: 0.0664 - categorical_accuracy: 0.9888 - precision_5: 0.9933 - recall_5: 0.9888 - val_loss: 0.1068 - val_categorical_accuracy: 0.9732 - val_precisi

Epoch 82/300
112/112 [==============================] - 8s 74ms/step - loss: 0.0286 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0438 - val_categorical_accuracy: 0.9911 - val_precision_5: 1.0000 - val_recall_5: 0.9911
Epoch 83/300
112/112 [==============================] - 8s 73ms/step - loss: 0.0249 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0397 - val_categorical_accuracy: 1.0000 - val_precision_5: 1.0000 - val_recall_5: 0.9911
Epoch 84/300
112/112 [==============================] - 8s 70ms/step - loss: 0.0229 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0598 - val_categorical_accuracy: 0.9911 - val_precision_5: 0.9911 - val_recall_5: 0.9911
Epoch 85/300
112/112 [==============================] - 8s 67ms/step - loss: 0.0315 - categorical_accuracy: 0.9978 - precision_5: 0.9978 - recall_5: 0.9955 - val_loss: 0.1174 - val_categorical_accuracy: 0.9643 - val_precisi

112/112 [==============================] - 7s 59ms/step - loss: 0.0071 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0434 - val_categorical_accuracy: 0.9821 - val_precision_5: 0.9821 - val_recall_5: 0.9821
Epoch 114/300
112/112 [==============================] - 6s 52ms/step - loss: 0.0068 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0470 - val_categorical_accuracy: 0.9821 - val_precision_5: 0.9821 - val_recall_5: 0.9821
Epoch 115/300
112/112 [==============================] - 6s 51ms/step - loss: 0.0066 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.0521 - val_categorical_accuracy: 0.9821 - val_precision_5: 0.9821 - val_recall_5: 0.9821
Epoch 116/300
5/5 [==============================] - 2s 20ms/step - loss: 0.0463 - categorical_accuracy: 0.9929 - precision_5: 1.0000 - recall_5: 0.9857


In [14]:
print("Loss:", loss)
print("Accuracies", accuracies)
print("Precisions", precisions)
print("Recalls", recalls)

Loss: [0.254977285861969, 0.10463964939117432, 0.055072955787181854, 0.21700255572795868, 0.04631892219185829]
Accuracies [0.9428571462631226, 0.9714285731315613, 0.9928571581840515, 0.9285714030265808, 0.9928571581840515]
Precisions [0.9492753744125366, 0.9784172773361206, 0.9928571581840515, 0.9285714030265808, 1.0]
Recalls [0.9357143044471741, 0.9714285731315613, 0.9928571581840515, 0.9285714030265808, 0.9857142567634583]


In [15]:
print("Mean Loss:", sum(loss)/len(loss))
print("Mean Accuracies", sum(accuracies)/len(accuracies))
print("Mean Precisions", sum(precisions)/len(precisions))
print("Mean Recalls", sum(recalls)/len(recalls))

Mean Loss: 0.13560227379202844
Mean Accuracies 0.9657142877578735
Mean Precisions 0.969824242591858
Mean Recalls 0.9628571391105651


In [16]:
model.save('lstmhand.h5')